In [364]:
import os
os.chdir('/home/ubuntu/code/pump_post_midterm/pump')

In [365]:
# # Get test_q_keys
# import random
# import pandas as pd
# from src.utils import get_file_from_s3

# survey_name = "American_Trends_Panel_W34"
# random.seed(42)
# survey_df = pd.read_csv(get_file_from_s3(f"human_resp/{survey_name}/info.csv"))
# q_keys = list(survey_df['key'])
# test_q_keys = random.choices(q_keys, k=5)

# print(test_q_keys)

In [366]:
import random
random.seed(42)

from src.utils import TEST_KEY_MAPPING

test_q_keys_mapping = TEST_KEY_MAPPING

# test_q_keys_mapping = {
#     "American_Trends_Panel_W26": ['GUNRESPNOKIDSB_W26', 'WORLDDANGER_W26', 'GUNIDENTITY_W26', 'REASONGUNC_W26', 'GUNRESPKIDSC_W26'],
#     "American_Trends_Panel_W27": ['CAREGIV3A_W27', 'PREDICTA_W27', 'WORK3C_W27', 'CARS10D_W27', 'CAREGIV7_W27'],
#     "American_Trends_Panel_W29": ['TRAITSD_W29', 'NOWSMK_NHIS_W29', 'HELPHURTE_W29', 'HELPHURTA_W29', 'GIRLSF2A_W29'],
#     "American_Trends_Panel_W32": ['NEIGHINTERB_W32', 'SATLIFEC_W32', 'SUCCESSIMPA_W32', 'COMMIMPE_W32', 'GROWUPUSR_W32'],
#     "American_Trends_Panel_W34": ['MED4A_W34', 'SCI2A_W34', 'FUD22_W34', 'EAT3H_W34', 'MED6D_W34']
# }


# Hyper parameters

In [367]:

survey_names = [
    # "American_Trends_Panel_W26",
    # "American_Trends_Panel_W27",
    # "American_Trends_Panel_W29",
    # "American_Trends_Panel_W32",
    "American_Trends_Panel_W34",
    "American_Trends_Panel_W41",
    "American_Trends_Panel_W82",
]

def skip_exp(filename):

    # check known test
    if 'v17' not in filename or all([_ not in filename for _ in ['0904', '0905', '0909', '0911']]): return True
    # if 'history_demo' in filename and 'history_demo_persona' not in filename: return True
    if 'bdemo' not in filename: return True
    # if 'bn' not in filename: return True

    return False

def get_exp(filename):
    if filename[-5:] == '.json': filename = filename[:-5]
    

    segs = ['history', 'demo', 'persona', 'withname', 'all' 'allq', 'ragupper', 'bdemo', 'rag', 'mostcommon', 'mostcommon40', 'bn', 'sonnetpred', 'date0911']
    for level in ['13', '16', '17']: segs.append('v'+level)
    for train in ['200', '500', '2000', 'All']: segs.append('train'+train)
    for top in range(10, 110, 10): segs.append('top'+str(top))
    for skew in ['5', '3', '10']: segs.append('skew'+skew)
    for usedemo in ['True', 'False']: segs.append('usedemo'+usedemo)
    for run in ['run1', 'run2', 'run3']: segs.append(run)
    for setting in ['queryponly', 'querydp', 'querydonly']: segs.append(setting)

    exp_elements = [_ for _ in filename.split('_') if _ in segs]
    final = '_'.join(exp_elements)
    return final

# Main

In [368]:
import json
import os
import pandas as pd
import numpy as np
import pandas as pd
import random

from collections import Counter
from src.utils import get_file_from_s3

def get_full_results(survey_name, sort_by):

    with open(f'experiment/data/human_resp/{survey_name}/user_test_q_key_mapping.json', 'r') as f:
        user_test_q_key_mapping = json.load(f)
    
    res = {}
    for filename in os.listdir(f'opinions_qa/output/{survey_name}/'):
        if skip_exp(filename): continue
        exp = get_exp(filename)
        # exp = filename
        if exp not in res:
            res[exp] = []
            
        with open(f"opinions_qa/output/{survey_name}/{filename}", 'r') as f:
            data = json.load(f)
        res[exp] += [f"={sum([_['is_correct'] for _ in data])}/{len(data)}", sum([_['is_correct'] for _ in data])/len(data)]

    for key in res:
        print(key, len(res[key]))

    df = pd.DataFrame(res)
    df = df[sorted(df.columns)]
    df = df.transpose()

    acc_df = df
    acc_df = acc_df.rename(columns={0: "acc_cnt", 1: "acc"})
    # acc_df.sort_values(by='acc', ascending=False)


    survey_df = pd.read_csv(get_file_from_s3(f"human_resp/{survey_name}/info.csv"))

    wd_question_mappings = {row['key']: dict(zip(eval(row['references'].lower()), eval(row['option_ordinal']))) for _, row in survey_df.iterrows()}
    wd_question_mappings

    records = {}
    for filename in os.listdir(f'opinions_qa/output/{survey_name}/'):
        if skip_exp(filename): continue
        # print(filename)
        with open(f"opinions_qa/output/{survey_name}/{filename}", 'r') as f:
            data = json.load(f)
        
        exp = get_exp(filename)

        res = []
        pred_missing_cnt = 0
        gold_missing_cnt = 0
        for entry in data:
            q_idx = entry['q_idx']
            q_key = user_test_q_key_mapping[str(entry["user_idx"])][q_idx]
            pos_mapping = wd_question_mappings[q_key]
            prediction = entry['prediction'].lower()
            gold_answer = entry['gold_answer'].lower()
            if gold_answer not in pos_mapping:
                gold_missing_cnt += 1
                continue
            if prediction not in pos_mapping:
                pred_missing_cnt += 1
                if prediction not in entry['references']:
                    refs = '/'.join(entry['references'].split('\n'))
                    print(f"Prediction: |{prediction}|, Gold: |{gold_answer}|, References: |{refs}|")
                continue
                prediction = random.choice(list(pos_mapping.keys()))
            pred_pos = pos_mapping[prediction]
            gold_pos = pos_mapping[gold_answer]
            res.append(abs(pred_pos-gold_pos))
        records[exp] = [Counter(res), np.mean(res), pred_missing_cnt, gold_missing_cnt]

    q_wd_df = pd.DataFrame(records)
    q_wd_df = q_wd_df[sorted(q_wd_df.columns)].transpose()
    q_wd_df = q_wd_df.rename(columns={0: "q_1wd_cnt", 1: "q_1wd", 2: "q_1wd_pmc", 3: "q_1wd_gmc"})
    q_wd_df

    # print(acc_df)
    # print(q_wd_df)

    merged_df = pd.merge(acc_df, q_wd_df, left_index=True, right_index=True)
    # merged_df = pd.merge(merged_df, s_wd_df, left_index=True, right_index=True)
    # merged_df = merged_df[['wd_avg', 'wd_std', 'acc_avg', 'acc_std']]

    if sort_by == 'q_1wd':
        merged_df = merged_df.sort_values(by='q_1wd', ascending=True)
    elif sort_by == 'acc':
        merged_df = merged_df.sort_values(by='acc', ascending=False)

    # merged_df.sort_values(by='acc_avg', ascending=False)
    # merged_df.sort_values(by='wd_avg', ascending=True)
    # merged_df = merged_df.sort_values(by='s_1wd', ascending=True)

    merged_df['acc_cnt'] = merged_df['acc_cnt'].apply(lambda x: '\''+str(x))
    merged_df
    print('\n\n\n')
    print(merged_df.to_csv())

    return merged_df

# Results

In [369]:
from IPython.display import display

for survey_name in survey_names:
    print(survey_name)
    sort_by = 'acc'
    # sort_by = 'q_1wd'
    merged = get_full_results(survey_name, sort_by=sort_by)
    display(merged)
    print('\n\n\n')

American_Trends_Panel_W34
v17_bdemo_history_demo_persona_run1 2
v17_bdemo_history_run1 2
v17_bdemo_history_demo_run2 2
v17_bdemo_history_demo_persona_rag_run2 2
v17_bdemo_mostcommon40_history_demo_persona_rag_run2 2
v17_bdemo_history_demo_run3 2
v17_bdemo_history_demo_persona_run2 2
v17_bdemo_bn_history_demo_persona_run3 2
v17_bdemo_mostcommon40_history_demo_persona_rag_run3 2
v17_bdemo_mostcommon_history_demo_persona_rag_run3 2
v17_bdemo_mostcommon40_bn_sonnetpred_history_demo_persona_rag_run3 2
v17_bdemo_mostcommon40_bn_history_demo_persona_rag_run2 2
v17_bdemo_mostcommon40_history_demo_persona_rag_run1 2
v17_bdemo_bn_history_demo_persona_run1 2
v17_bdemo_mostcommon_history_demo_persona_rag_run1 2
v17_bdemo_mostcommon40_bn_sonnetpred_history_demo_persona_rag_run1 2
v17_bdemo_mostcommon_history_demo_persona_rag_run2 2
v17_bdemo_history_demo_persona_rag_run3 2
v17_bdemo_bn_history_demo_persona_run2 2
v17_bdemo_history_demo_persona_rag_run1 2
v17_bdemo_mostcommon40_bn_history_demo_perso

,acc_cnt,acc,q_1wd_cnt,q_1wd,q_1wd_pmc,q_1wd_gmc
v17_bdemo_mostcommon40_bn_history_demo_persona_rag_run2,'=624/1000,0.624,"{0.0: 623, 1.0: 307, 2.0: 17, 3.0: 1, 0.5: 12}",0.364583,27,13
v17_bdemo_mostcommon40_bn_history_demo_persona_rag_run1,'=621/1000,0.621,"{0.0: 620, 1.0: 311, 2.0: 18, 3.0: 1, 0.5: 12}",0.370062,25,13
v17_bdemo_mostcommon40_bn_sonnetpred_history_demo_persona_rag_run2,'=620/1000,0.62,"{0.0: 620, 1.0: 333, 2.0: 20, 3.0: 1, 0.5: 12}",0.387424,1,13
v17_bdemo_mostcommon40_bn_sonnetpred_history_demo_persona_rag_run3,'=620/1000,0.62,"{0.0: 620, 1.0: 333, 2.0: 20, 3.0: 1, 0.5: 12}",0.387424,1,13
v17_bdemo_mostcommon40_bn_history_demo_persona_rag_run3,'=620/1000,0.62,"{0.0: 620, 1.0: 312, 2.0: 18, 3.0: 1, 0.5: 13}",0.370851,23,13
v17_bdemo_mostcommon40_bn_sonnetpred_history_demo_persona_rag_run1,'=619/1000,0.619,"{0.0: 619, 1.0: 334, 2.0: 20, 3.0: 1, 0.5: 12}",0.388438,1,13
v17_bdemo_mostcommon40_history_demo_persona_rag_run2,'=619/1000,0.619,"{0.0: 619, 1.0: 317, 2.0: 15, 0.5: 13, 3.0: 1}",0.36943,22,13
v17_bdemo_mostcommon40_history_demo_persona_rag_run3,'=612/1000,0.612,"{0.0: 612, 1.0: 318, 2.0: 16, 0.5: 13, 3.0: 1}",0.374479,27,13
v17_bdemo_mostcommon40_history_demo_persona_rag_run1,'=611/1000,0.611,"{0.0: 611, 1.0: 320, 2.0: 16, 0.5: 13, 3.0: 1}",0.376171,26,13
v17_bdemo_mostcommon_history_demo_persona_rag_run3,'=605/1000,0.605,"{0.0: 605, 1.0: 320, 2.0: 20, 3.0: 1, 0.5: 9}",0.384817,32,13






American_Trends_Panel_W41
v17_bdemo_history_run1 2
v17_bdemo_history_demo_persona_run3 2
v17_bdemo_history_demo_run1 2
v17_bdemo_history_demo_run3 2
v17_bdemo_mostcommon40_history_demo_persona_rag_run1 2
v17_bdemo_mostcommon40_history_demo_persona_rag_run3 2
v17_bdemo_mostcommon40_bn_history_demo_persona_rag_run1 2
v17_bdemo_history_run2 2
v17_bdemo_history_demo_run2 2
v17_bdemo_mostcommon40_history_demo_persona_rag_run2 2
v17_bdemo_mostcommon40_bn_history_demo_persona_rag_run3 2
v17_bdemo_history_demo_persona_run1 2
v17_bdemo_history_run3 2
v17_bdemo_mostcommon40_bn_history_demo_persona_rag_run2 2
v17_bdemo_history_demo_persona_run2 2
Prediction: |outside of a professional or work setting|, Gold: |yes|, References: |Yes/No/Refused|
Prediction: |maintained at current levels|, Gold: |an important, but not a top priority|, References: |A top priority/An important, but not a top priority/A lower priority/Should not be done/Refused|
Prediction: |both|, Gold: |yes|, References: |Yes/No/

,acc_cnt,acc,q_1wd_cnt,q_1wd,q_1wd_pmc,q_1wd_gmc
v17_bdemo_mostcommon40_bn_history_demo_persona_rag_run3,'=559/1000,0.559,"{0.0: 570, 1.0: 282, 0.5: 79, 2.0: 40, 3.0: 3,...",0.430244,2,16
v17_bdemo_mostcommon40_history_demo_persona_rag_run2,'=559/1000,0.559,"{0.0: 569, 1.0: 295, 0.5: 73, 2.0: 37, 1.5: 4,...",0.433876,1,16
v17_bdemo_mostcommon40_bn_history_demo_persona_rag_run2,'=557/1000,0.557,"{0.0: 568, 1.0: 282, 0.5: 81, 2.0: 39, 3.0: 4,...",0.432281,2,16
v17_bdemo_mostcommon40_history_demo_persona_rag_run3,'=555/1000,0.555,"{0.0: 566, 1.0: 297, 0.5: 73, 2.0: 37, 1.5: 5,...",0.437436,1,16
v17_bdemo_mostcommon40_bn_history_demo_persona_rag_run1,'=554/1000,0.554,"{0.0: 565, 1.0: 285, 0.5: 80, 2.0: 41, 3.0: 3,...",0.435845,2,16
v17_bdemo_mostcommon40_history_demo_persona_rag_run1,'=553/1000,0.553,"{0.0: 563, 1.0: 301, 0.5: 73, 2.0: 37, 1.5: 5,...",0.441057,0,16
v17_bdemo_history_demo_persona_run1,'=517/1000,0.517,"{1.0: 328, 0.0: 529, 0.5: 72, 2.0: 40, 1.5: 5,...",0.471341,7,16
v17_bdemo_history_demo_persona_run2,'=514/1000,0.514,"{1.0: 330, 0.0: 526, 0.5: 72, 2.0: 42, 1.5: 5,...",0.476994,6,16
v17_bdemo_history_demo_persona_run3,'=513/1000,0.513,"{1.0: 331, 0.0: 525, 0.5: 75, 2.0: 40, 1.5: 5,...",0.474974,5,16
v17_bdemo_history_demo_run2,'=513/1000,0.513,"{0.0: 526, 1.0: 332, 0.5: 79, 2.0: 40, 1.5: 2,...",0.469419,3,16






American_Trends_Panel_W82
v17_bdemo_history_demo_run1 2
v17_bdemo_history_run3 2
v17_bdemo_mostcommon40_bn_history_demo_persona_rag_run3 2
v17_bdemo_history_demo_persona_run2 2
v17_bdemo_history_run1 2
v17_bdemo_mostcommon40_history_demo_persona_rag_run3 2
v17_bdemo_mostcommon40_bn_history_demo_persona_rag_run1 2
v17_bdemo_history_demo_persona_run1 2
v17_bdemo_history_demo_run2 2
v17_bdemo_mostcommon40_history_demo_persona_rag_run1 2
v17_bdemo_mostcommon40_bn_history_demo_persona_rag_run2 2
v17_bdemo_history_run2 2
v17_bdemo_history_demo_run3 2
v17_bdemo_mostcommon40_history_demo_persona_rag_run2 2
v17_bdemo_history_demo_persona_run3 2
Prediction: |very confident|, Gold: |a lot of confidence|, References: |A lot of confidence/Some confidence/Not too much confidence/No confidence at all/Refused|
Prediction: |very confident|, Gold: |a lot of confidence|, References: |A lot of confidence/Some confidence/Not too much confidence/No confidence at all/Refused|
Prediction: |in foreign poli

,acc_cnt,acc,q_1wd_cnt,q_1wd,q_1wd_pmc,q_1wd_gmc
v17_bdemo_mostcommon40_bn_history_demo_persona_rag_run3,'=585/1000,0.585,"{0.0: 585, 1.0: 343, 2.0: 37, 0.5: 10, 3.0: 4}",0.443309,2,19
v17_bdemo_mostcommon40_bn_history_demo_persona_rag_run1,'=583/1000,0.583,"{0.0: 583, 1.0: 343, 2.0: 38, 0.5: 10, 3.0: 3}",0.443193,4,19
v17_bdemo_mostcommon40_history_demo_persona_rag_run1,'=579/1000,0.579,"{0.0: 578, 1.0: 347, 2.0: 38, 0.5: 11, 3.0: 4}",0.450409,3,19
v17_bdemo_mostcommon40_history_demo_persona_rag_run2,'=579/1000,0.579,"{0.0: 579, 1.0: 343, 2.0: 38, 0.5: 11, 3.0: 4}",0.447692,6,19
v17_bdemo_mostcommon40_bn_history_demo_persona_rag_run2,'=578/1000,0.578,"{0.0: 578, 1.0: 347, 2.0: 39, 0.5: 10, 3.0: 3}",0.449335,4,19
v17_bdemo_mostcommon40_history_demo_persona_rag_run3,'=578/1000,0.578,"{0.0: 578, 1.0: 346, 2.0: 39, 0.5: 11, 3.0: 4}",0.451431,3,19
v17_bdemo_history_demo_persona_run1,'=575/1000,0.575,"{0.0: 570, 1.0: 353, 2.0: 37, 0.5: 11, 3.0: 3}",0.453285,7,19
v17_bdemo_history_demo_persona_run2,'=575/1000,0.575,"{0.0: 572, 1.0: 354, 2.0: 36, 0.5: 11, 3.0: 3}",0.451332,5,19
v17_bdemo_history_demo_persona_run3,'=574/1000,0.574,"{0.0: 570, 1.0: 356, 2.0: 35, 0.5: 11, 3.0: 3}",0.451795,6,19
v17_bdemo_history_demo_run3,'=573/1000,0.573,"{0.0: 571, 1.0: 358, 2.0: 36, 0.5: 8, 3.0: 5}",0.4591,3,19


In [370]:
,acc_cnt,acc,q_1wd_cnt,q_1wd,q_1wd_pmc,q_1wd_gmc
v17_bdemo_mostcommon40_bn_history_demo_persona_rag_run3,'=585/1000,0.585,"Counter({0.0: 585, 1.0: 343, 2.0: 37, 0.5: 10, 3.0: 4})",0.4433094994892748,2,19
v17_bdemo_mostcommon40_bn_history_demo_persona_rag_run1,'=583/1000,0.583,"Counter({0.0: 583, 1.0: 343, 2.0: 38, 0.5: 10, 3.0: 3})",0.44319344933469806,4,19
v17_bdemo_mostcommon40_history_demo_persona_rag_run1,'=579/1000,0.579,"Counter({0.0: 578, 1.0: 347, 2.0: 38, 0.5: 11, 3.0: 4})",0.4504089979550102,3,19
v17_bdemo_mostcommon40_history_demo_persona_rag_run2,'=579/1000,0.579,"Counter({0.0: 579, 1.0: 343, 2.0: 38, 0.5: 11, 3.0: 4})",0.44769230769230767,6,19
v17_bdemo_mostcommon40_bn_history_demo_persona_rag_run2,'=578/1000,0.578,"Counter({0.0: 578, 1.0: 347, 2.0: 39, 0.5: 10, 3.0: 3})",0.44933469805527126,4,19
v17_bdemo_mostcommon40_history_demo_persona_rag_run3,'=578/1000,0.578,"Counter({0.0: 578, 1.0: 346, 2.0: 39, 0.5: 11, 3.0: 4})",0.4514314928425358,3,19
v17_bdemo_history_demo_persona_run1,'=575/1000,0.575,"Counter({0.0: 570, 1.0: 353, 2.0: 37, 0.5: 11, 3.0: 3})",0.45328542094455854,7,19
v17_bdemo_history_demo_persona_run2,'=575/1000,0.575,"Counter({0.0: 572, 1.0: 354, 2.0: 36, 0.5: 11, 3.0: 3})",0.45133196721311475,5,19
v17_bdemo_history_demo_persona_run3,'=574/1000,0.574,"Counter({0.0: 570, 1.0: 356, 2.0: 35, 0.5: 11, 3.0: 3})",0.4517948717948718,6,19
v17_bdemo_history_demo_run3,'=573/1000,0.573,"Counter({0.0: 571, 1.0: 358, 2.0: 36, 0.5: 8, 3.0: 5})",0.4591002044989775,3,19
v17_bdemo_history_run2,'=571/1000,0.571,"Counter({0.0: 570, 1.0: 349, 2.0: 38, 0.5: 10, 3.0: 4})",0.4552008238928939,10,19
v17_bdemo_history_demo_run1,'=570/1000,0.57,"Counter({0.0: 568, 1.0: 361, 2.0: 36, 0.5: 9, 3.0: 5})",0.46220633299284986,2,19
v17_bdemo_history_demo_run2,'=568/1000,0.568,"Counter({0.0: 566, 1.0: 361, 2.0: 36, 0.5: 8, 3.0: 5})",0.46311475409836067,5,19
v17_bdemo_history_run3,'=567/1000,0.567,"Counter({0.0: 566, 1.0: 350, 2.0: 40, 0.5: 11, 3.0: 4})",0.46086508753862,10,19
v17_bdemo_history_run1,'=561/1000,0.561,"Counter({0.0: 560, 1.0: 361, 2.0: 38, 0.5: 10, 3.0: 4})",0.46659815005138744,8,19


SyntaxError: unterminated string literal (detected at line 2) (885638539.py, line 2)